In [1]:
#%pip install requests
#%pip install bs4

In [2]:
from bs4 import BeautifulSoup # type: ignore
import requests # type: ignore
import pandas as pd # type: ignore
import urllib.parse
#import os
#from PIL import Image
#from IPython.display import IFrame

In [3]:
import numpy as np

In [4]:
#%pip install selenium

In [5]:
from selenium import webdriver # type: ignore
from selenium.webdriver.chrome.options import Options # type: ignore
from selenium.webdriver.chrome.service import Service # type: ignore
from selenium.webdriver.support.ui import WebDriverWait # type: ignore
from selenium.common.exceptions import TimeoutException # type: ignore
from time import sleep

In [6]:
#url_get = 'https://shopee.com.my/search?'
#payload = {"keyword":"dashcam"}
#r = requests.get(url_get, params=payload)
#print(r.url)
#print(r.status_code)
#print(r.content)

In [7]:
#base_url = 'https://shopee.com.my/search?keyword=dashcam&trackingId=searchhint-1728046904-ce774037-8250-11ef-8e22-3ea6d88eabcb'
#base_url = 'https://shopee.com.my/shop/13377506/search?page=0&sortBy=sales'
#base_url = 'https://www.lazada.com.my/tag/dashcam/?spm=a2o4k.homepage.search.d_go&q=dashcam&catalog_redirect_tag=true'
base_url = 'https://www.lazada.com.my/tag/dashcam/?catalog_redirect_tag=true&page=1&q=dashcam&spm=a2o4k.homepage.search.d_go'
chrome_options = Options()

# set chrome driver options to disable any popup's from the website
# to find local path for chrome profile, open chrome browser
# and in the address bar type, "chrome://version"
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\1\\AppData\\Local\\Google\\Chrome\\User Data\\Default')
# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")
# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 2
    })
# invoke the webdriver
chrome_service = Service(executable_path = r"C:\Users\1\OneDrive\Desktop\Personal Matter\Data Analytics Learning\Self Exploration Project\chromedriver-win64\chromedriver.exe")
browser = webdriver.Chrome(service = chrome_service, options = chrome_options)

#browser.get(base_url)

delay = 5 #seconds

In [8]:
item_cost, item_init_cost, item_loc = [],[],[]
item_name, items_sold, discount_percent = [], [], []

for i in range(40):
    page_num = i + 1
    base_url = 'https://www.lazada.com.my/tag/dashcam/?catalog_redirect_tag=true&page=' + str(page_num) + '&q=dashcam&spm=a2o4k.homepage.search.d_go'
    browser.get(base_url)
    
    while True:
        try:
            WebDriverWait(browser, delay)
            print ("Page ", str(page_num), " is ready")
            sleep(5)
            html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
            #print(html)
            soup = BeautifulSoup(html, "html.parser")

            # find_all() returns an array of elements. 
            # We have to go through all of them and select that one you are need. And than call get_text()
            for item_n in soup.find_all('div', class_='RfADt'):
                #print(item_n.get_text())
                item_name.append(item_n.text)

            # find the price of items
            for item_c in soup.find_all('span', class_='ooOxS'):
                #print(item_c.get_text())
                item_cost.append(item_c.text)

            # find initial item cost
            #for item_ic in soup.find_all('div', class_ = '_1w9jLI QbH7Ig U90Nhh'):
            #    print(item_ic.get_text())
            #    item_init_cost.append(item_ic.text)
            
            # find total number of items sold/month
            for items_s in soup.find_all('span',class_ = '_1cEkb'):
                #print(items_s.get_text())
                items_sold.append(items_s.text)

            # find item discount percent
            for dp in soup.find_all('span', class_ = 'ic-dynamic-badge ic-dynamic-badge-text ic-dynamic-badge-120015 ic-dynamic-group-2'):
                #print(dp.get_text())
                discount_percent.append(dp.text)

            # find item location
            #for il in soup.find_all('div', class_ = '_3amru2'):
            #    print(il.get_text())
            #    item_loc.append(il.text)

            break # it will break from the loop once the specific element will be present. 
        except TimeoutException:
            print ("Loading took too much time!-Try again")

# close the automated browser
browser.close()

Page  1  is ready
Page  2  is ready
Page  3  is ready
Page  4  is ready
Page  5  is ready
Page  6  is ready
Page  7  is ready
Page  8  is ready
Page  9  is ready
Page  10  is ready
Page  11  is ready
Page  12  is ready
Page  13  is ready
Page  14  is ready
Page  15  is ready
Page  16  is ready
Page  17  is ready
Page  18  is ready
Page  19  is ready
Page  20  is ready
Page  21  is ready
Page  22  is ready
Page  23  is ready
Page  24  is ready
Page  25  is ready
Page  26  is ready
Page  27  is ready
Page  28  is ready
Page  29  is ready
Page  30  is ready
Page  31  is ready
Page  32  is ready
Page  33  is ready
Page  34  is ready
Page  35  is ready
Page  36  is ready
Page  37  is ready
Page  38  is ready
Page  39  is ready
Page  40  is ready


In [9]:
#rows = zip(item_name, item_cost, discount_percent, items_sold)

In [27]:
import csv
newFilePath1 = 'lazada_item_list_1600.csv'
newFilePath2 = 'lazada_item_list_800.csv'
#with open(newFilePath, "w") as f:
#    writer = csv.writer(f)
#    for row in rows:
#        writer.writerow(row)

In [11]:
df = pd.DataFrame(columns= ['product_name', 'price_RM', 'discount_percent', 'quantity_sold'])
df['product_name'] = pd.Series(item_name)
df['price_RM'] = pd.Series(item_cost)
df['discount_percent'] = pd.Series(discount_percent)
df['quantity_sold'] = pd.Series(items_sold)

df['price_RM'] = df['price_RM'].replace(['RM', ","], '', regex=True)
df['price_RM'] = df['price_RM'].astype(float)

df['discount_percent'] = df['discount_percent'].replace(to_replace=np.nan, value='Voucher save 0%')
df['discount_percent'] = df['discount_percent'].replace(to_replace="Voucher save ", value="", regex=True)
df['discount_percent'] = df['discount_percent'].replace(to_replace="%", value="", regex=True)
df['discount_percent'] = df['discount_percent'].astype(int)

df['original_price_RM'] = df['price_RM'] / (1 - (df['discount_percent'] / 100))
df['original_price_RM'] = df['original_price_RM'].round(2)
df['original_price_RM'] = df['original_price_RM'].astype(float)

df['quantity_sold'] = df['quantity_sold'].replace(' sold', '', regex=True)
df['quantity_sold'] = df['quantity_sold'].replace('K', '000', regex=True)
df['quantity_sold'] = df['quantity_sold'].replace(np.nan, '0')
df['quantity_sold'] = df['quantity_sold'].astype(str).str.replace(".", "")
df['quantity_sold'] = df['quantity_sold'].astype(int)

print(df.dtypes)
df.head()

product_name          object
price_RM             float64
discount_percent       int64
quantity_sold          int64
original_price_RM    float64
dtype: object


,product_name,price_RM,discount_percent,quantity_sold,original_price_RM
0,DOSEN S50 Driving Recorder Front interior Rear...,135.80,50,228,271.60
1,DOSNE A1 Driving Recorder Front interior Rear ...,79.80,43,323,140.00
2,DOSNE Wifi 行车记录仪 24 Hours Record 1080P FHD wif...,79.80,43,341,140.00
3,DOSEN A1 wireless Driving Recorder Front inter...,79.66,60,31000,199.15
4,DOSEN 24 Hours Record 1080P FHD Wifi 3 Camera ...,79.66,58,60000,189.67


In [25]:
df_800 = df[df.index < 800]
df_800.tail()

,product_name,price_RM,discount_percent,quantity_sold,original_price_RM
795,Dashcam Car Reverse Image Video Recorder G-sen...,77.07,58,36,183.50
796,4K 3840*2160P Dash Cam Mini WIFI GPS Car DVR C...,203.32,29,52,286.37
797,4K 2160P Single Len Car DVR Dash Cam Rear View...,77.35,8,7,84.08
798,3-Channel Car DVR 1080P Dash Cam for Cars WIFI...,53.65,33,123,80.07
799,Car DVR 10.26 Inch Dash Cam 4K 2160P Carplay A...,324.96,23,22,422.03


In [28]:
df.to_csv(newFilePath1)
df_800.to_csv(newFilePath2)

In [22]:
df.describe()

,price_RM,discount_percent,quantity_sold,original_price_RM
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,193.504081,20.348125,2574.877500,272.563656
std,179.614480,23.956745,16652.662283,315.422734
min,3.000000,0.000000,0.000000,3.610000
25%,73.090000,0.000000,0.000000,89.900000
50%,130.905000,3.000000,6.000000,181.365000
75%,256.762500,42.000000,32.250000,335.827500
max,1679.000000,88.000000,240000.000000,4825.000000


In [29]:
df_800.describe()

,price_RM,discount_percent,quantity_sold,original_price_RM
count,800.000000,800.0000,800.000000,800.000000
mean,180.858662,40.2975,5042.862500,338.062050
std,176.130551,18.4089,23208.097778,396.385338
min,3.000000,1.0000,5.000000,3.610000
25%,68.915000,27.0000,11.000000,118.012500
50%,119.180000,42.0000,27.000000,215.215000
75%,239.000000,53.0000,167.000000,422.837500
max,1679.000000,88.0000,240000.000000,4825.000000


In [13]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [30]:
# Checking dashcam price distribution in boxplot
fig1 = go.Figure()
fig1.add_trace(go.Box(y=df_800['original_price_RM']))
fig1.update_layout(title_text='<b>Dashcam Price Distribution</b>')
fig1.update_yaxes(title_text='Price (RM)')

fig1.show()

In [53]:
# Checking dashcam price distribution in histogram
fig2 = go.Figure()
fig2.add_trace(go.Histogram(x=df_800['original_price_RM'], nbinsx=64))
fig2.update_layout(title_text='<b>Dashcam Price Distribution</b>')
fig2.update_xaxes(title_text='Price (RM)')
fig2.update_yaxes(title_text="Count")

fig2.show()

In [52]:
# Checking dashcam price vs quantity sold distribution in histogram
fig2_1 = go.Figure()
fig2_1.add_trace(go.Histogram(histfunc='avg', x=df_800['original_price_RM'], y=df['quantity_sold'], nbinsx=64))
fig2_1.update_layout(title_text='<b>Dashcam Price vs Average Quantity Sold Distribution</b>')
fig2_1.update_xaxes(title_text="Price (RM)")
fig2_1.update_yaxes(title_text="Average Quantity Sold")

fig2_1.show()

In [37]:
# Checking correlation between price and sales
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=df_800['original_price_RM'], y=df_800['quantity_sold'], mode='markers'))
fig3.update_layout(title_text='<b>Dashcam Price vs Quantity Sold on Lazada</b>')
fig3.update_xaxes(title_text="Price (RM)")
fig3.update_yaxes(title_text="Quantity Sold")

fig3.show()

In [39]:
# Checking correlation between discount and sales
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=df_800['discount_percent'], y=df_800['quantity_sold'], mode='markers'))
fig4.update_layout(title_text='<b>Dashcam Discount % vs Quantity Sold on Lazada</b>')
fig4.update_xaxes(title_text="Discount %")
fig4.update_yaxes(title_text="Quantity Sold")

fig4.show()

In [54]:
df_above_500 = df_800[df_800['original_price_RM'] >= 500]
df_above_500.tail()

,product_name,price_RM,discount_percent,quantity_sold,original_price_RM
754,Ezonetronics cheap 9.35 Full Touch Screen HD M...,295.05,51,19,602.14
770,[NEW] 70mai A810 4K Dash Cam Dual Vision Car R...,609.00,12,8,692.05
780,EKLEVA 3 Channel WiFi Dash Cam For Cars DVR Ca...,192.70,68,63,602.19
790,DDPAI Z40 1994P Front n Rear DashCam Recorder ...,598.00,3,14,616.49
791,IROAD DASH CAM TX-1 FULL HD WITH TOUCH SCREEN,530.00,42,30,913.79


In [55]:
# Checking correlation between discount and sales for dashcam above RM 500
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=df_above_500['discount_percent'], y=df_above_500['quantity_sold'], mode='markers'))
fig5.update_layout(title_text='<b>Dashcam (> RM 500) Discount % vs Quantity Sold on Lazada</b>')
fig5.update_xaxes(title_text="Discount %")
fig5.update_yaxes(title_text="Quantity Sold")

fig5.show()

In [56]:
from plotly.subplots import make_subplots # type: ignore

In [62]:
# Dashboard
dashfig1_2 = make_subplots(
    rows=1, cols=2,
    subplot_titles=("<b>Dashcam Price Distribution</b>",
                    "<b>Dashcam Price vs Average Quantity Sold Distribution</b>"))

dashfig1_2.add_trace(go.Histogram(x=df_800['original_price_RM'],
                                  nbinsx=64,
                                  showlegend=False),
                                  row=1,
                                  col=1)
dashfig1_2.add_trace(go.Histogram(histfunc='avg',
                                  x=df_800['original_price_RM'],
                                  y=df['quantity_sold'],
                                  nbinsx=64,
                                  showlegend=False),
                                  row=1,
                                  col=2)

dashfig1_2.update_layout(autosize=False,
                      width=1200,
                      height=500,
                      margin=dict(l=100, r=150, b=50, t=100))

dashfig1_2.update_xaxes(title_text="Price (RM)", row=1, col=1)
dashfig1_2.update_xaxes(title_text='Price (RM)', row=1, col=2)

dashfig1_2.update_yaxes(title_text="Count", row=1, col=1)
dashfig1_2.update_yaxes(title_text="Average Quantity Sold", row=1, col=2)

dashfig1_2.show()

In [63]:
dashfig3_4 = make_subplots(
    rows=1, cols=2,
    subplot_titles=("<b>Dashcam Price vs Quantity Sold</b>",
                    "<b>Dashcam Discount % vs Quantity Sold</b>"))

dashfig3_4.add_trace(go.Scatter(x=df_800['original_price_RM'],
                                y=df_800['quantity_sold'],
                                mode='markers',
                                showlegend=False),
                                row=1,
                                col=1)
dashfig3_4.add_trace(go.Scatter(x=df_800['discount_percent'],
                                y=df_800['quantity_sold'],
                                mode='markers',
                                showlegend=False),
                                row=1,
                                col=2)

dashfig3_4.update_layout(autosize=False,
                      width=1200,
                      height=500,
                      margin=dict(l=100, r=150, b=100, t=50))

dashfig3_4.update_xaxes(title_text="Price (RM)", row=1, col=1)
dashfig3_4.update_xaxes(title_text="Discount %", row=1, col=2)

dashfig3_4.update_yaxes(title_text="Quantity Sold", row=1, col=1)
dashfig3_4.update_yaxes(title_text="Quantity Sold", row=1, col=2)

dashfig3_4.show()

In [64]:
dashfigs = [dashfig1_2, dashfig3_4]

dashboard = open("Dashcam Prices on Lazada Dashboard.html", "w")
dashboard.write("<html><body>" + "\n")
dashboard.write("<head> <link href='https://fonts.googleapis.com/css?family=Open Sans' rel='stylesheet'>" + "\n")
dashboard.write("<link href='styles.css' rel='stylesheet'></head>" + "\n")
dashboard.write("<h1>Newborn Trend in Malaysia</h1>" + "\n")
dashboard.write("<p1>Sample taken: first 800 from product search result with keyword 'dashcam' on Lazada</p1>" + "\n")
for fig in dashfigs:
    dashboard.write(fig.to_html(full_html=False, include_plotlyjs='cdn') + "\n")
dashboard.write("<p2>this project is for personal learning purpose only</p2>" + "\n")
dashboard.write("</body></html>")
dashboard.close()